### The OpenMDAO way
---
#### Create a `Component` to represent the Rosenbrock function

In [1]:
import numpy as np
import openmdao.api as om
    
import time
    
ORDER = 6  # dimension of problem
span = 2   # upper and lower limits

class RosenbrockComp(om.ExplicitComponent):
    def setup(self):
        self.add_input('x', np.zeros(ORDER))
        self.add_output('y', 0.0)

    def compute(self, inputs, outputs):
        x = inputs['x']

        n = len(x)
        s = 0
        for i in range(n - 1):
            s += 100 * (x[i + 1] - x[i] * x[i]) ** 2 + (1 - x[i]) ** 2

        outputs['y'] = s

#### Create a model by putting the `Component` in a `Group` and specifying the design variables and objective

In [2]:
my_model = om.Group()
my_model.add_subsystem('rosenbrock', RosenbrockComp())
my_model.add_design_var('rosenbrock.x', lower=-span * np.ones(ORDER), upper=span * np.ones(ORDER))
my_model.add_objective('rosenbrock.y')

#### Create a `Problem` to optimize the model using your choice of optimizers as the `Driver`

In [3]:
p = om.Problem(model=my_model, driver=om.DifferentialEvolutionDriver(max_gen=800))
p.setup()

start = time.time()
fail = p.run_driver()
elapsed = time.time() - start

assert fail is False

#### The global minimum is found at [1, 1, 1, ...]

In [4]:
print('xopt:', p['rosenbrock.x'])
print('fopt:', p['rosenbrock.y'])
print('iterations:', p.driver.iter_count)
print('elapsed time:', elapsed)

xopt: [1.00000036 1.00000038 1.00000064 1.00000073 1.00000215 1.00000444]
fopt: [9.95555983e-11]
iterations: 96121
elapsed time: 79.99333810806274





### The `pycma` way
---
#### Using the functional interface

In [5]:
import cma

# the Rosenbrock function is available in the "fitness functions" library
rosenbrock = cma.ff.rosen

# find global minimum via the fmin function
start = time.time()
res = cma.fmin(rosenbrock, ORDER * [-1], 0.01,
               options={'ftarget':1e-6, 'verb_time':0, 'verb_disp':100, 'seed':3},
               restarts=3)
elapsed = time.time() - start

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=3, Sun Oct 18 23:09:46 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 2.001843015092397e+03 1.0e+00 8.78e-03  8e-03  9e-03 
    2     18 1.986125694620930e+03 1.4e+00 8.48e-03  8e-03  9e-03 
    3     27 1.988866857733969e+03 1.5e+00 7.68e-03  7e-03  8e-03 
  100    900 2.372114162584607e+00 8.1e+00 8.61e-02  1e-02  5e-02 
  200   1800 1.088872415974996e-01 2.6e+01 1.29e-01  6e-03  4e-02 
  266   2394 6.637172252308916e-07 5.3e+01 4.33e-03  5e-05  9e-04 
termination on ftarget=1e-06 (Sun Oct 18 23:09:48 2020)
final/bestever f-value = 6.720343e-07 6.637172e-07
incumbent solution: [0.999982151058333, 0.9999866455805502, 0.9999825497485988, 0.9999100175543002, 0.9998658254305461, 0.9997588042973703]
std deviation: [5.3875484003553336e-05, 7.218132721595697e-05, 0.00011048916508625072, 0.0002242804956513507, 0.0004619327772433858, 0.0009076162191461554]


#### The global minimum is found at [1, 1, 1, ...]

In [6]:
xopt = res[0]
fopt = res[1]
evalsopt = res[2]
evals = res[3]
iterations = res[4]
print('xopt:', res[0])
print('fopt:', res[1])
print('evalsopt:', res[2])
print('evals:', res[3])
print('iterations:', res[4])
print('elapsed time:', elapsed)

xopt: [0.99997403 0.99994719 0.9999355  0.9998314  0.99970865 0.99942561]
fopt: 6.637172252308916e-07
evalsopt: 2390
evals: 2395
iterations: 266
elapsed time: 1.526317834854126


### Use `pycma` as top level interface over OpenMDAO
---
#### Wrap an OpenMDAO `Component` for use with the `pycma` functional interface

In [7]:
comp = RosenbrockComp()

def rosenbrock(x):
    inputs = {'x': x}
    outputs = {}
    comp.compute(inputs, outputs)
    return outputs['y']
    
start = time.time()
res = cma.fmin(rosenbrock, ORDER * [-1], 0.01,
               options={'ftarget':1e-6, 'verb_time':0, 'verb_disp':100, 'seed':3},
               restarts=3)
elapsed = time.time() - start

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=3, Sun Oct 18 23:09:48 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 2.001843015092397e+03 1.0e+00 8.78e-03  8e-03  9e-03 
    2     18 1.986125694620930e+03 1.4e+00 8.48e-03  8e-03  9e-03 
    3     27 1.988866857733969e+03 1.5e+00 7.68e-03  7e-03  8e-03 
  100    900 2.372114162584607e+00 8.1e+00 8.61e-02  1e-02  5e-02 
  200   1800 1.088872415974996e-01 2.6e+01 1.29e-01  6e-03  4e-02 
  266   2394 6.637172252308916e-07 5.3e+01 4.33e-03  5e-05  9e-04 
termination on ftarget=1e-06 (Sun Oct 18 23:09:49 2020)
final/bestever f-value = 6.720343e-07 6.637172e-07
incumbent solution: [0.999982151058333, 0.9999866455805502, 0.9999825497485988, 0.9999100175543002, 0.9998658254305461, 0.9997588042973703]
std deviation: [5.3875484003553336e-05, 7.218132721595697e-05, 0.00011048916508625072, 0.0002242804956513507, 0.0004619327772433858, 0.0009076162191461554]


In [8]:
xopt = res[0]
fopt = res[1]
evalsopt = res[2]
evals = res[3]
iterations = res[4]
print('xopt:', res[0])
print('fopt:', res[1])
print('evalsopt:', res[2])
print('evals:', res[3])
print('iterations:', res[4])
print('elapsed time:', elapsed)

xopt: [0.99997403 0.99994719 0.9999355  0.9998314  0.99970865 0.99942561]
fopt: 6.637172252308916e-07
evalsopt: 2390
evals: 2395
iterations: 266
elapsed time: 1.3665874004364014


### Use `pycma` as top level interface over OpenMDAO
---
#### Wrap an OpenMDAO `Problem` for use with the `pycma` functional interface

In [11]:
def rosenbrock(x):
    p['rosenbrock.x'] = x
    p.run_model()
    return p['rosenbrock.y']
    
start = time.time()
res = cma.fmin(rosenbrock, ORDER * [-1], 0.01,
               options={'ftarget':1e-6, 'verb_time':0, 'verb_disp':100, 'seed':3},
               restarts=3)
elapsed = time.time() - start

(4_w,9)-aCMA-ES (mu_w=2.8,w_1=49%) in dimension 6 (seed=3, Sun Oct 18 23:11:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      9 2.001843015092397e+03 1.0e+00 9.87e-03  9e-03  1e-02 
termination on tolfun=1e-11 (Sun Oct 18 23:11:25 2020)
final/bestever f-value = 2.029508e+03 2.001843e+03
incumbent solution: [-0.9893478469085577, -0.9976819186390338, -1.0003795523127295, -1.0113174525315063, -1.0049002512864622, -1.0015264314443975]
std deviation: [0.010049127438745819, 0.009410691177377204, 0.009610223792995162, 0.010524533675087874, 0.00965023559197976, 0.009749170101372575]
(9_w,18)-aCMA-ES (mu_w=5.4,w_1=30%) in dimension 6 (seed=4, Sun Oct 18 23:11:25 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     28 2.001283718740498e+03 1.0e+00 9.00e-03  8e-03  9e-03 
termination on tolfun=1e-11 after 1 restart (Sun Oct 18 23:11:26 2020)
final/bestever f-value = 2.021054e+03 2.001284e+03
incumbent solution: [-0.99753

In [10]:
xopt = res[0]
fopt = res[1]
evalsopt = res[2]
evals = res[3]
iterations = res[4]
print('xopt:', res[0])
print('fopt:', res[1])
print('evalsopt:', res[2])
print('evals:', res[3])
print('iterations:', res[4])
print('elapsed time:', elapsed)

xopt: [-0.99949438 -0.99500045 -1.00995926 -0.99306384 -1.00418315 -1.01584643]
fopt: 2001.2837187404984
evalsopt: 11
evals: 139
iterations: 1
elapsed time: 0.3718082904815674
